In [2]:
import pyspark
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.master("local[*]").appName('week5').getOrCreate()

your 131072x1 screen size is bogus. expect trouble
24/03/06 13:03:48 WARN Utils: Your hostname, dashel resolves to a loopback address: 127.0.1.1; using 172.31.179.36 instead (on interface eth0)
24/03/06 13:03:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/06 13:03:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Question 1

In [4]:
spark.version

'3.5.0'

### Question 2

In [5]:
df_fhv_2019_10 = spark.read.option('header', 'true').csv('fhv_tripdata_2019-10.csv.gz')

In [6]:
df_fhv_2019_10.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B00009|2019-10-01 00:23:00|2019-10-01 00:35:00|         264|         264|   NULL|                B00009|
|              B00013|2019-10-01 00:11:29|2019-10-01 00:13:22|         264|         264|   NULL|                B00013|
|              B00014|2019-10-01 00:11:43|2019-10-01 00:37:20|         264|         264|   NULL|                B00014|
|              B00014|2019-10-01 00:56:29|2019-10-01 00:57:47|         264|         264|   NULL|                B00014|
|              B00014|2019-10-01 00:23:09|2019-10-01 00:28:27|         264|         264|   NULL|                B00014|
|     B00021         |2019-10-01 00:00:4

In [7]:
df_fhv_2019_10.schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropOff_datetime', StringType(), True), StructField('PUlocationID', StringType(), True), StructField('DOlocationID', StringType(), True), StructField('SR_Flag', StringType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [8]:
from pyspark.sql import types

In [9]:
schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True), 
    types.StructField('pickup_datetime', types.TimestampType(), True), 
    types.StructField('dropOff_datetime', types.TimestampType(), True), 
    types.StructField('PUlocationID', types.IntegerType(), True), 
    types.StructField('DOlocationID', types.IntegerType(), True), 
    types.StructField('SR_Flag', types.StringType(), True),
    types.StructField('Affiliated_base_number', types.StringType(), True)
])

In [10]:
df_fhv_2019_10 = spark.read.option('header', 'true').schema(schema).csv('fhv_tripdata_2019-10.csv.gz')

In [12]:
df_fhv_2019_10.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B00009|2019-10-01 00:23:00|2019-10-01 00:35:00|         264|         264|   NULL|                B00009|
|              B00013|2019-10-01 00:11:29|2019-10-01 00:13:22|         264|         264|   NULL|                B00013|
|              B00014|2019-10-01 00:11:43|2019-10-01 00:37:20|         264|         264|   NULL|                B00014|
|              B00014|2019-10-01 00:56:29|2019-10-01 00:57:47|         264|         264|   NULL|                B00014|
|              B00014|2019-10-01 00:23:09|2019-10-01 00:28:27|         264|         264|   NULL|                B00014|
|     B00021         |2019-10-01 00:00:4

In [14]:
df_fhv_2019_10.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropOff_datetime: timestamp (nullable = true)
 |-- PUlocationID: integer (nullable = true)
 |-- DOlocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [15]:
df_fhv_2019_10.repartition(6).write.parquet('parquet/2019/10')

In [16]:
!ls -lh parquet/2019/10/

total 39M
-rwxrwxrwx 1 dashelruiz dashelruiz    0 Mar  6 15:49 _SUCCESS
-rwxrwxrwx 1 dashelruiz dashelruiz 6.4M Mar  6 15:49 part-00000-3705e195-66dd-4c1f-888e-3d32981af095-c000.snappy.parquet
-rwxrwxrwx 1 dashelruiz dashelruiz 6.4M Mar  6 15:49 part-00001-3705e195-66dd-4c1f-888e-3d32981af095-c000.snappy.parquet
-rwxrwxrwx 1 dashelruiz dashelruiz 6.4M Mar  6 15:49 part-00002-3705e195-66dd-4c1f-888e-3d32981af095-c000.snappy.parquet
-rwxrwxrwx 1 dashelruiz dashelruiz 6.4M Mar  6 15:49 part-00003-3705e195-66dd-4c1f-888e-3d32981af095-c000.snappy.parquet
-rwxrwxrwx 1 dashelruiz dashelruiz 6.4M Mar  6 15:49 part-00004-3705e195-66dd-4c1f-888e-3d32981af095-c000.snappy.parquet
-rwxrwxrwx 1 dashelruiz dashelruiz 6.4M Mar  6 15:49 part-00005-3705e195-66dd-4c1f-888e-3d32981af095-c000.snappy.parquet


- Answer 6.4M

### Question 3

In [19]:
from pyspark.sql import functions as F

In [21]:
df = spark.read.parquet('parquet/2019/10/*')

In [23]:
df.registerTempTable('fhv')

/home/dashelruiz/anaconda3/envs/de_env/lib/python3.12/site-packages/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [26]:
spark.sql("""
SELECT COUNT(*) AS total_records
FROM
    fhv
WHERE
    DATE(pickup_datetime) = '2019-10-15'
"""
).show()

+-------------+
|total_records|
+-------------+
|        62610|
+-------------+



### Question 4

In [33]:
from pyspark.sql.functions import col, asc, desc

In [29]:
df.head(1)

[Row(dispatching_base_num='B01843', pickup_datetime=datetime.datetime(2019, 10, 29, 16, 49), dropOff_datetime=datetime.datetime(2019, 10, 29, 20, 29), PUlocationID=264, DOlocationID=264, SR_Flag=None, Affiliated_base_number=None)]

In [34]:
df \
    .withColumn('duration_seconds', df.dropOff_datetime.cast('long')-df.pickup_datetime.cast('long')) \
    .orderBy(col('duration_seconds').desc()) \
    .show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+----------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|duration_seconds|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+----------------+
|              B02832|2019-10-28 09:00:00|2091-10-28 09:30:00|         264|         264|   NULL|                B02832|      2272149000|
|              B02832|2019-10-11 18:00:00|2091-10-11 18:30:00|         264|         264|   NULL|                B02832|      2272149000|
|              B02416|2019-10-31 23:46:33|2029-11-01 00:13:00|        NULL|        NULL|   NULL|                B02416|       315620787|
|     B00746         |2019-10-01 21:43:42|2027-10-01 21:45:23|         159|         264|   NULL|       B00746         |       252460901|
|              B02921|2019-10-17 14:00:00

In [36]:
df.columns

# method 1
df \
    .withColumn('duration', df.dropOff_datetime.cast('long') - df.pickup_datetime.cast('long')) \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .groupBy('pickup_date') \
        .max('duration') \
    .orderBy('max(duration)', ascending=False) \
    .limit(5) \
    .show()

+-----------+-------------+
|pickup_date|max(duration)|
+-----------+-------------+
| 2019-10-28|   2272149000|
| 2019-10-11|   2272149000|
| 2019-10-31|    315620787|
| 2019-10-01|    252460901|
| 2019-10-17|     31658400|
+-----------+-------------+



- Answer = 631,152.50 

### Question 5

- Answer 4040

### Questin 6

In [40]:
df_zones = spark.read.parquet('zones/')

In [41]:
df_zones.head(1)

[Row(LocationID='1', Borough='EWR', Zone='Newark Airport', service_zone='EWR')]

In [42]:
df_zones.registerTempTable('zones')

/home/dashelruiz/anaconda3/envs/de_env/lib/python3.12/site-packages/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)
